**ИЗУЧЕНИЕ НАБОРА ДАННЫХ О СОСТОЯНИИ ЗДОРОВЬЯ ЛОШАДЕЙ**

Изучим закономерности в наборе данных о состоянии здоровья лошадей, испытывающих кишечные колики для корректного заполнения пропусков и выявления основных закономерностей.

Изучаемый набор данных содержит характеристики, связанные со здоровьем животного. Объекты имеют 3 типа исходов - "умерла", "подверглась эвтаназии" и "жива".

* Умерла:  что лошадь умерла из-за проблем со здоровьем.

* Эвтаназия: гуманное умерщвление лошади. Иногда тяжелые болезни и несчастные случаи причиняют лошадям страдания. Владельцы добровольно подвергают таких животных эвтаназии, часто путем физического разрушения мозга выстрелами.

* Выжила: Это означает, что лошадь либо выжила после болезни, либо никогда не имела проблем со здоровьем. Более подробную информацию мы можем получить во время EDA.

Некоторые медицинские термины

* Ректальная температура: Нормальный диапазон ректальной температуры лошади составляет 37,5-38,6ºC.

* Пульс: У взрослых лошадей пульс в норме составляет от 28 до 40 ударов в минуту. У новорожденных жеребят частота пульса колеблется от 80 до 120, у жеребят постарше - от 60 до 80, а у годовалых лошадей - от 40 до 60 ударов в минуту.

* Частота дыхания: Нормальная частота дыхания для лошади составляет от 8 до 16 вдохов в минуту.

* Температура конечностей: Это температура частей тела, удаленных от основной части тела, таких как ноги, копыта и т.д.

* Периферический пульс: Периферический пульс означает пальпацию волны высокого давления крови, отходящей от сердца по сосудам конечностей.





Для исследования выберем признаки surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [241]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

horses = pd.read_csv('horse_data.txt')

In [243]:

full_list=['surgery?', 'age', 'Hospital Number', 'rectal temperature', 'pulse', 'respiratory rate', 
                          'temperature of extremities', 'peripheral pulse', 'mucous membranes', 'capillary refill time', 
                          'pain', 'peristalsis', 'abdominal distension', 'nasogastric tube', 'nasogastric reflux',
                          'nasogastric reflux PH', 'rectal examination - feces', 'abdomen', 'packed cell volume',
                          'total protein', 'abdominocentesis appearance', 'abdomcentesis total protein', 'outcome',
                          'surgical lesion?', '25', '26', '27', 'cp_data']
selection_list=['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']  
full_series=pd.Series(full_list)                
usecols=full_series[full_series.isin(selection_list)]                 

In [245]:
horse=pd.read_csv('horse_data.txt', header=None, usecols=usecols.index, na_values='?', names=usecols)
horse.head()

,surgery?,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,38.5,66.0,28.0,3.0,5.0,2.0
1,1.0,39.2,88.0,20.0,NaN,3.0,3.0
2,2.0,38.3,40.0,24.0,1.0,3.0,1.0
3,1.0,39.1,164.0,84.0,4.0,2.0,2.0
4,2.0,37.3,104.0,35.0,NaN,NaN,2.0


В нашем распоряжении данные следующих видов:

1) Номинальные бинарные: surgery?

2) Интервальные: rectal temperature, 	pulse, 	respiratory rate 	

3) Порядковые: temperature of extremities, 	pain. 

В первом признаке похоже нарушен порядок кодировки (перепутан порядок warm и normal). В таком виде данный признак превращается в категориальный, и должен быть векторизирован. Альтернативно, мы можем сгрупировать классы данного признака и бинаризировать на пары "normal+warm"/"cool+cold". Поскольку уточнения в задании отсутствуют, данный параметр оставлен без изменения.

4) Категориальные: 	outcome (Строго говоря могут быть разбиты на 2 бинарных признака: 'lived' и 'died/'euthanized')

В данном ноутбуке преобразовываем только признак surgery?

In [247]:
horse['surgery?']=np.where(horse['surgery?']==1, True, False)


In [248]:
horse.columns

Index(['surgery?', 'rectal temperature', 'pulse', 'respiratory rate',
       'temperature of extremities', 'pain', 'outcome'],
      dtype='object')


**Первичное изучение данных**

Начинаем наше исследование с анализа значения по столбцам, рассчёта основных статистик и определения выбросов

Для начала выясним, насколько связаны между собою наблюдения сгрупированные по исходам. Для этого проанализируем эти подвыборки методом Manova.

In [252]:
import pandas as pd
from statsmodels.multivariate.manova import MANOVA
horses=horse.copy()
horses.columns=horses.columns.str.replace(" ", "")
horses.columns=horses.columns.str.replace("?", "")

In [253]:
maov = MANOVA.from_formula('surgery + rectaltemperature + pulse + respiratoryrate  + pain ~ outcome', data=horses.query('outcome!=2'))
print(maov.mv_test())

                  Multivariate linear model
                                                             
-------------------------------------------------------------
       Intercept        Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.0000 6.0000 122.0000     inf 0.0000
         Pillai's trace 1.0000 6.0000 122.0000     inf 0.0000
 Hotelling-Lawley trace    inf 6.0000 122.0000     inf 0.0000
    Roy's greatest root    inf 6.0000 122.0000     inf 0.0000
-------------------------------------------------------------
                                                             
-------------------------------------------------------------
        outcome         Value  Num DF  Den DF  F Value Pr > F
-------------------------------------------------------------
          Wilks' lambda 0.8244 5.0000 123.0000  5.2384 0.0002
         Pillai's trace 0.1756 5.0000 123.0000  5.2384 0.0002
 Hotelling-Lawley trace 0.

C:\ProgramData\Anaconda3\envs\NEW\lib\site-packages\statsmodels\multivariate\multivariate_ols.py:168: RuntimeWarning: divide by zero encountered in scalar divide
  eigv1 = np.array([i / (1 - i) for i in eigv2])
C:\ProgramData\Anaconda3\envs\NEW\lib\site-packages\statsmodels\multivariate\multivariate_ols.py:168: RuntimeWarning: invalid value encountered in scalar divide
  eigv1 = np.array([i / (1 - i) for i in eigv2])
C:\ProgramData\Anaconda3\envs\NEW\lib\site-packages\statsmodels\multivariate\multivariate_ols.py:199: RuntimeWarning: divide by zero encountered in scalar divide
  F = (1 - lmd) / lmd * df2 / df1
C:\ProgramData\Anaconda3\envs\NEW\lib\site-packages\statsmodels\multivariate\multivariate_ols.py:209: RuntimeWarning: divide by zero encountered in scalar divide
  F = df2 / df1 * V / (s - V)


In [254]:
maov = MANOVA.from_formula('surgery + rectaltemperature + pulse + respiratoryrate  + pain ~ outcome', data=horses)
print(maov.mv_test())

                                 Multivariate linear model
                                                                                           
-------------------------------------------------------------------------------------------
       Intercept                Value         Num DF  Den DF         F Value         Pr > F
-------------------------------------------------------------------------------------------
          Wilks' lambda               -0.0000 6.0000 162.0000 -5066549580791835.0000 1.0000
         Pillai's trace                1.0000 6.0000 162.0000 -5066549580791835.0000 1.0000
 Hotelling-Lawley trace -187649984473771.6562 6.0000 162.0000 -5066549580791835.0000 1.0000
    Roy's greatest root -187649984473771.6562 6.0000 162.0000 -5066549580791835.0000 1.0000
-------------------------------------------------------------------------------------------
                                                                                           
---------------------

In [255]:
maov = MANOVA.from_formula('surgery + rectaltemperature + pulse + respiratoryrate  + pain ~ outcome', data=horses.query('outcome!=1'))
print(maov.mv_test())

                               Multivariate linear model
                                                                                       
---------------------------------------------------------------------------------------
       Intercept               Value         Num DF  Den DF       F Value        Pr > F
---------------------------------------------------------------------------------------
          Wilks' lambda               0.0000 6.0000 55.0000 971364625511274.3750 0.0000
         Pillai's trace               1.0000 6.0000 55.0000 971364625511274.1250 0.0000
 Hotelling-Lawley trace 105967050055775.3750 6.0000 55.0000 971364625511274.2500 0.0000
    Roy's greatest root 105967050055775.3750 6.0000 55.0000 971364625511274.2500 0.0000
---------------------------------------------------------------------------------------
                                                                                       
---------------------------------------------------------------

По данным Pr > F, различия между исходом 1 и исходами 2+3 значимы, а исходы 2 и 3 (умершие и усыплённые) статистически неразличимы (возможно результаты после нормализации данных будут отличаться).

Поэтому, учитывая категориальный характер переменной outcome, мы будем рассматривать данные в разрезе исходов.

Рассмотрим как меняются статистические данные по группам выживаемости/смертности

In [258]:
statoverview=horse.groupby('outcome').agg(['count','min', 'max', 'mean', 'median']).round(1)
statoverview.T

outcome                              1.0    2.0    3.0
surgery?                   count     178     77     44
                           min     False  False  False
                           max      True   True   True
                           mean      0.5    0.8    0.6
                           median    1.0    1.0    1.0
rectal temperature         count     152     53     34
                           min      36.5   36.0   35.4
                           max      39.9   40.8   40.3
                           mean     38.2   38.2   38.1
                           median   38.2   38.1   38.0
pulse                      count     166     66     43
                           min      30.0   30.0   36.0
                           max     150.0  184.0  140.0
                           mean     63.4   87.2   81.7
                           median   54.0   86.0   82.0
respiratory rate           count     147     58     36
                           min       8.0   12.0   12.0
                           max      96.0   96.0   80.0
                           mean     29.3   33.0   31.1
                           median   24.0   30.0   24.0
temperature of extremities count     146     64     33
                           min       1.0    1.0    1.0
                           max       4.0    4.0    4.0
                           mean      2.0    2.8    2.9
                           median    2.0    3.0    3.0
pain                       count     144     65     35
                           min       1.0    1.0    1.0
                           max       5.0    5.0    5.0
                           mean      2.5    3.7    3.3
                           median    3.0    4.0    3.0

Судя по данным:

1) Хирургическое вмешательство чаще проводилось в случае выживших и усыплённых.

2) Для умерших и усыплённых животных медианное значение пульса значительно выше чем для выживших. Похоже это самый значимо различающийся по значениям признак (необходима дальнейшая проверка).

3) Медианная частота дыхания умерших животных выше чем у выживших и усыплённых.

4) Температура конечностей для выживших животных медианно тёплая, для остальных групп медианно прохладная.

5) Медианная болезненность выше для умерших животный


Для начала определим признаки, в которых содержатся нетипичные значения. Это три интервальные переменные 'rectal temperature', 'pulse', 'respiratory rate'.
Определим выбросы методом Тьюки для каждого исхода, а потом найдём выбросы по полной выборке. 

In [261]:
quant_v=1.5 # 
outcoms=['lived', 'died', 'euthed']
confdict=dict()
for i in [0, 1, 2]:
    horse_group= horse[horse.outcome==i+1].describe().round(1).T
    # display(horse_group)
    IQR=horse_group['75%']-horse_group['25%']
    CI_left=horse_group['25%']-quant_v*IQR
    CI_right=horse_group['75%']+quant_v*IQR
    confdict[outcoms[i]]=pd.DataFrame([CI_left, CI_right], index=['left', 'right'])
lived= confdict['lived']
died=confdict['died']
euthed=confdict['euthed']
feat_dest_confint=pd.concat(confdict).T.iloc[:-1, :]
feat_dest_confint

lived           died         euthed        
                             left   right   left   right   left   right
rectal temperature          36.75   39.55  36.05   40.45  35.75   40.15
pulse                       12.00  108.00  18.75  144.75   0.00  160.00
respiratory rate           -13.25   64.75   6.00   54.00 -10.00   70.00
temperature of extremities  -2.00    6.00   3.00    3.00   3.00    3.00
pain                         0.50    4.50   0.00    8.00  -1.00    7.00

Мы видим что у совокупости выживших особей отмечаются более узкие доверительные интервалы ректальных температур, более низкие верхние границы доверительного интервала пульса и более высокое верхнее значение доверительного интервала показателя температуры конечности. Также у группы выживших лошадей отмечалась более низкая верхняя граница доверительного интервала оценки интенсивности болевого синдрома.

**Поиск выбросный значений**

In [264]:
Посмотрим, имеются ли у нас выбросные значения.

SyntaxError: invalid syntax (1312282455.py, line 1)

In [ ]:
rt_ci=lived.loc['left':'right', 'rectal temperature']
pulse_ci=lived.loc['left':'right', 'pulse']
rr_ci=lived.loc['left':'right','respiratory rate']
rt_outliers=np.invert(horse.query('outcome==1')['rectal temperature'].between(rt_ci.left, rt_ci.right))
pulse_outliers=np.invert(horse.query('outcome==1')['pulse'].between(pulse_ci.left, pulse_ci.right))
rr_outliers=np.invert(horse.query('outcome==1')['respiratory rate'].between(rr_ci.left, rr_ci.right))

all_feat_ol_lived=(rt_outliers&pulse_outliers&rr_outliers)
print(all_feat_ol_lived.value_counts())
horse.iloc[all_feat_ol_lived[all_feat_ol_lived].index]

In [ ]:
# rt_outliers=horse.query('outcome==1')['rectal temperature'].between(rt_ci.left, rt_ci.right) 
# pulse_outliers=horse.query('outcome==1')['pulse'].between(pulse_ci.left, pulse_ci.right)
# rr_outliers=horse.query('outcome==1')['respiratory rate'].between(rr_ci.left, rr_ci.right)

# all_feat_nol_lived=(rt_outliers&pulse_outliers&rr_outliers)
# print(all_feat_nol_lived.value_counts())
# horse.iloc[all_feat_nol_lived[~all_feat_nol_lived].index]

Выбросы по всем трём параметрам у нас наблюдаются в 8 случаях. 

Для выздоровевших лошадей в качестве выбросов найдены только отсутствующие значения. Причина в отсутствии выбросов в измерениях ректальной температуры. повторим проверку без этого признака.

In [ ]:
fig,ax=plt.subplots(2, 1,  figsize=(18, 6))
sns.boxplot(data=horse.query('outcome==1')[['pulse']], ax=ax[0], orient = 'h', whis=1.5).grid(True)
sns.boxplot(data=horse.query('outcome==1')[['respiratory rate']], ax=ax[1], orient = 'h').grid(True)
plt.show()
all_feat_ol_lived=(pulse_outliers&rr_outliers)
print(all_feat_ol_lived.value_counts())
horse.iloc[all_feat_ol_lived[all_feat_ol_lived].index]


Для выживших лошадей мы нашли выбросные (по признакам pulse 	respiratory rate) наблюдения №№: 41, 103, 148*, 229, 244.


*-По данным одного лишь признака pulse

Повторим проверку на выбросы для остальных классов лошадиной судьбы

In [ ]:
fig,ax=plt.subplots(2, 1,  figsize=(18, 6))
sns.boxplot(data=horse.query('outcome==2')[['pulse']], ax=ax[0], orient = 'h').grid(True)
sns.boxplot(data=horse.query('outcome==2')[['respiratory rate']], ax=ax[1], orient = 'h').grid(True)
plt.show()
rt_ci=died.loc['left':'right', 'rectal temperature']
pulse_ci=died.loc['left':'right', 'pulse']
rr_ci=died.loc['left':'right','respiratory rate']
pulse_outliers=np.invert(horse.query('outcome==2')['pulse'].between(pulse_ci.left, pulse_ci.right))
rr_outliers=np.invert(horse.query('outcome==2')['respiratory rate'].between(rr_ci.left, rr_ci.right))

all_feat_ol_died=(pulse_outliers&rr_outliers)
print(all_feat_ol_died.value_counts())
horse.iloc[all_feat_ol_died[all_feat_ol_died].index]

Для умерших лошадей мы нашли выбросные (по выбранным ранее признакам) наблюдения №№: 3, 39, 255

Для умерщвленных лошадей:


In [ ]:
fig,ax=plt.subplots(2, 1,  figsize=(18, 6))
sns.boxplot(data=horse.query('outcome==3')[['pulse']], ax=ax[0], orient = 'h', whis=1.5).grid(True)
sns.boxplot(data=horse.query('outcome==3')[['respiratory rate']], ax=ax[1], orient = 'h').grid(True)
plt.show()
pulse_ci=euthed.loc['left':'right', 'pulse']
rr_ci=euthed.loc['left':'right','respiratory rate']
pulse_outliers=np.invert(horse.query('outcome==3')['pulse'].between(pulse_ci.left, pulse_ci.right))
rr_outliers=np.invert(horse.query('outcome==3')['respiratory rate'].between(rr_ci.left, rr_ci.right))
all_feat_ol_euthed=(pulse_outliers&rr_outliers)
print(all_feat_ol_euthed.value_counts())
horse.iloc[all_feat_ol_euthed[all_feat_ol_euthed].index]

Для усыплённых лошадей выбросные наблюдения не найдены.

Наконец примененим метод Тьюки к полной выборке

In [ ]:
statoverview=horse.describe().round(1).T
IQR=statoverview['75%']-statoverview['25%']
CI_left=statoverview['25%']-quant_v*IQR
CI_right=statoverview['75%']+quant_v*IQR
conframe=pd.DataFrame([CI_left, CI_right], index=['left', 'right'])
conframe.T

Рассмотрим полученные результаты. 

In [ ]:
fig,ax=plt.subplots(2, 1,  figsize=(18, 6))
sns.boxplot(data=horse[['pulse']], ax=ax[0], orient = 'h', whis=1.5).grid(True)
sns.boxplot(data=horse[['respiratory rate']], ax=ax[1], orient = 'h').grid(True)
plt.show()
pulse_ci=conframe.loc['left':'right', 'pulse']
rr_ci=conframe.loc['left':'right','respiratory rate']

pulse_outliers=np.invert(horse['pulse'].between(pulse_ci.left, pulse_ci.right))
rr_outliers=np.invert(horse['respiratory rate'].between(rr_ci.left, rr_ci.right))

all_feat_ol_conframe=(pulse_outliers&rr_outliers)
print(all_feat_ol_conframe.value_counts())
# horse.iloc[all_feat_ol_conframe[all_feat_ol_conframe]]
horse[all_feat_ol_conframe]

На полной выборке метод Тьюки нашёл всего 3 выбросных значения, это наблюдения № 3, 41, 255.

Итак мы осуществили поиск выбросов на всей выборке и подвыборках сформированных по исходу лечения. Несмотря на то, что большинство найденых на множестве выживших лошадей подозрительных записей укладывается в диапазон характерных значений полной выборки, они могут отражать более тяжёлое протекание болезни которое имело определённые вероятности благополучного и неблагополучного исхода. Если это так, повышенное значение данных показателей могут влиять на прогноз.


**Заполнение пропусков**

Рассчитаем количество пропусков для всех выбранных столбцов. 

In [ ]:
feat_na=horse.isna().sum() # Определяем количество и пропусков по признакам
feat_na_pct=(feat_na/horse.shape[1]).round(0) 
pd.DataFrame([feat_na, feat_na_pct.round(2)],
             index=["Пропуски", "% Пропусков"]).astype(int).T 

In [ ]:
# Определяем количество и процент записей содержащих пропуски по количеству отсутствующих в них признаков
row_na=horse.iloc[:, horse.columns!='outcome'].isna().T.sum().value_counts(
    dropna=False)  
row_na_pct=row_na/horse.shape[1]
display(pd.DataFrame([row_na, 
              row_na_pct.round(2)],
              index=["Кол-во строк", 
                     "% строк с пропусками"]).T.astype(int).reset_index()
                     .rename(columns=({'index': "Кол-во пропусков"})))
plt.figure(figsize=[7, 7])
plt.pie(x=row_na, labels=row_na.index, autopct='%1.0f%%', 
        explode=[0, 0, 0, 0.2, 0.2, 0.2])
plt.show()

Лишь половина записей не имеет пропущенных значений, четверть имеет один пропуск и в каждой 6 записи отсутствуют 2 значения. Целесообразность рассмотрения сторк с бОльшим количеством отсутствующих значений признаков под вопросом, а это практически 8% строк датафрейма. Возможным решением является рассмотрение признака surgery? отдельно от остальных, однако 

Посмотрим для каких признаков ситуация неблагоприятная

In [ ]:
horse.loc[horse.isna().sum(axis=1)>=3].count().sort_values()

В 22 строках имеющих 3 и более пропусков лишь в двух случаях имеются значения для признаков rectal temperature и respiratory rate. 

In [ ]:
na_pct_feat=feat_na.sum()/horse.size
print('Всего пропусков по ячейкам датафейма: {}%'.format(round(na_pct_feat*100, 1)))

Рассмотрим полученные результаты. Записи в которых отсутствуют 4 и более признаков, неинформативны и их целесообразно удалить. Также удалим запись с неизвестным исходом. Сделаем это и повторно рассмотрим данные.

In [ ]:
horse=horse.loc[horse.isna().sum(axis=1)<4]
horse=horse[horse.outcome.notna()]
print(horse.isna().sum())
print('\n Количество наблюдений после удаления: {}'.format(horse.shape[0]))

Рассмотрим на корреляции признаков в разрезе исходов. Для начала посмотрим какие признаки связаны с исходом лечения, а далее оценим тесноту связи между независимыми признаками в разрезе исхода.

Учитывая наличие среди признаков категориальных и порядковых, для поиска корреляционных зависимостей на всём пространстве признаков мы будем использовать корреляцию Кенделла, а для корреляций между интервальными признаками мы также проведём корреляцию Пирсона. 



In [ ]:
horse.corr(method='kendall').outcome.round(2)

In [ ]:
horse.groupby('outcome').corr(method="kendall").round(2)

In [ ]:
horse.groupby('outcome')[['rectal temperature', 'pulse', 'respiratory rate']].corr(method="pearson").round(2)

По признакам:

1) **surgery?**: для выживших связь с болевым синдромом (у них видимо был более ярко выражен, поскольку организм по видимому покрепче), для умерщвлённых - с частотой дыхания (возможно неудачный исход операции).

2) **rectal temperature**: корреляция с пульсом для умерших животных и понижением тмпературы конечностей. 

3) **pulse**: высокая корреляция  (высокая) с частотой дыхания, с болезненностью для выживших,а для умерших и умерщвлённых со снижением температуры конечностей (особенно для последних). Таким образом выходит что температура конечностей более информативный признак для прогнозирования исхода. Также, важно снижение корреляции пульса и частоты дыхания в ряду выжившие-умершие-эвтанизированные, по всей видимости снижение за счёт тяжести  состояния. Сочетание этих признаков выглядит перспективно с точки зрения прогнозирования исхода лечения. Для умерших также имеется связь с ректальной температурой. 

Как мы видели ранее, медианные температуры одинаковы для всех исходов, поэтому заменяем медианным значением по всему датасету.

In [ ]:
horse['rectal temperature'].fillna(horse['rectal temperature'].median(), inplace=True)

Мы также видели, что медианный пульс у выживших и погибших животных сильно различается. Заменим отсутствующие значения модальными пульсами для исходов.
То же самое проделаем с иными признаками. Альтернативно, осуществим замены с применением алгоритма MICE

In [ ]:
horse_med=horse.copy()

In [ ]:
horse_med.shape

In [ ]:
horse_med.pulse=horse_med.groupby("outcome").pulse.transform(lambda x: x.fillna(x.median()))
horse_med['respiratory rate']=horse_med.groupby("outcome")['respiratory rate'].transform(lambda x: x.fillna(x.median()))
horse_med['temperature of extremities']=horse_med.groupby("outcome")['temperature of extremities'].transform(lambda x: x.fillna(x.median()))
horse_med['pain']=horse_med.groupby("outcome")['pain'].transform(lambda x: x.fillna(x.median()))

Попробуем теперь осуществить замену отсутствующих значений с помощью алгоритма MICE

In [ ]:
# Imputing with MICE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model

# Define MICE Imputer and fill missing values
mice_imputer = IterativeImputer(estimator=linear_model.BayesianRidge(), n_nearest_features=None, imputation_order='ascending')

horse_mice_imputed = pd.DataFrame(mice_imputer.fit_transform(horse), columns=horse.columns)

Определим насколько коррелированы наши датасеты после замены отсутствующих значений

In [ ]:
horse_med.reset_index(drop=True).corrwith(horse_mice_imputed.reset_index(drop=True))

In [ ]:
listtodf=[]
for i in range(1, 4):
    listtodf.append(horse_med[horse_med['outcome']==i].reset_index(drop=True).corrwith(horse_mice_imputed[horse_mice_imputed['outcome']==i].reset_index(drop=True)))
df=pd.DataFrame(listtodf).round(3)
df.index.name='outcome'
del df['outcome']
display(df)


ЗАКЛЮЧЕНИЕ

Датафрейм содержит достаточно много данных в сравнении с полным объёмом данных в исходном текстовом файле. Между некоторыми признаками имеется достаточно сильная корреляция, однако мультиколлинеарность маловероятна. Вполне ожидаемо подтерждается предположение о том, что во вличнии на выздоровление и летальный исход могут преобладать разные признаки.

Использованы две стратегии замены данных, и какая из них более адекватна задаче необходимо выяснить дополнительно. Также возможно имеет смысл объединить исходы 2 и 3, а естественность смерти животного закодировать отдельно. 




